In [2]:
import pandas as pd
import pickle

In [ ]:
### Carga df desde excel
path_xlsx = "/Users/mac2/Library/CloudStorage/GoogleDrive-edwar.perez@vlex.com/Mi unidad/EP_Coding_Vlex/Martindale/datos_abogados.xlsx"
df_0 = pd.read_excel(path_xlsx)
df_0 = df_0.dropna(axis=1, how='all')
df_0.iloc[:,9:].head(2)
#df_0.head(2)


In [95]:
### Carga del df desde un archivo .pkl
with open("data_attornies.pkl", "rb") as f:
    data_for_PA = pickle.load(f) # Carga de lista de diccionarios de abogados de primera iteración, ya la lista no es 0.
    # Esta variable va a ser mas general por lo que se va a emplear mas adelante. o asignar a la primera corrida. 
data_for_PA
df = pd.DataFrame(data_for_PA)
df = df.where(pd.notnull(df), None)
df = df.dropna(axis=1, how='all')
df.drop(['Education & Credentials'], axis=1, inplace=True)
df.head(5)


,url_martindale,name,firm,url_firm_martindale,website_firm,Biography,Areas of Practice,Contact Information,University Attended,Law School Attended,...,"Other Practice Areas in Carolina, Puerto Rico","Other Practice Areas in Bayamon, Puerto Rico","Other Practice Areas in Dorado, Puerto Rico","Other Practice Areas in Guayama, Puerto Rico","Other Practice Areas in Isabela, Puerto Rico","Other Practice Areas in San German, Puerto Rico","Other Practice Areas in Vega Baja, Puerto Rico","Other Practice Areas in Cabo Rojo, Puerto Rico","Other Practice Areas in Salinas, Puerto Rico","Other Practice Areas in Aguadilla, Puerto Rico"
0,https://www.martindale.com/attorney/tessie-lea...,"Tessie Leal-Garabis, Esq.",Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Tessie Leal Garabis is a trial attorney with o...,"Trial Practice, Civil Appeals, Business Law, M...",787-650-6994Phone787-767-2645Faxwww.mellado.co...,"Bently University, B.A., 1995","Pontifical Catholic University of Puerto Rico,...",...,None,None,None,None,None,None,None,None,None,None
1,https://www.martindale.com/attorney/ramon-mell...,Ramón Mellado-González,Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Ramón has been an attorney and notary in Puert...,"Complex Business Litigation, Banks and Banking...",787-650-6994Phone787-767-2645Faxwww.mellado.co...,"University of Puerto Rico, B.A., Political Sci...","University of Puerto Rico, LL.B., 1958",...,None,None,None,None,None,None,None,None,None,None
2,https://www.martindale.com/attorney/edgardo-j-...,"Edgardo J. Hernández Ohárriz, Esq.","Hernandez - Oharriz & Santiago Law Firm, P.S.C.",https://www.martindale.com/organization/hernan...,http://www.hoslawfirm.com,Partner/co-founder of Hernandez-Oharriz & Sant...,"Personal Injury, Pedestrian Injuries, Plaintif...",855-789-8177Phone787-775-1672Faxwww.hoslawfirm...,"University of Puerto Rico, B.B.A., 2007; Unive...","Inter-American University, J.D., 2010",...,None,None,None,None,None,None,None,None,None,None
3,https://www.martindale.com/attorney/jaime-j-ap...,Jaime J. Aponte-Parsi,"Jaime Aponte-Parsi Law Offices, P.S.C.",https://www.martindale.com/organization/jaime-...,http://www.aponte-parsi.com,Former Partner: Maymi & Aponte-Parsi (2000-200...,"Corporate Law, Taxation, Real Estate, Estate S...",877-792-1046Phone787-622-7669Faxwww.aponte-par...,"Boston College, B.A., Economics and Political ...","Boston College, J.D., 1978",...,None,None,None,None,None,None,None,None,None,None
4,https://www.martindale.com/attorney/vicente-a-...,Vicente A. Sequeda-Torres,"Reichard & Calaf, P.S.C.",https://www.martindale.com/organization/reicha...,http://reichardcalaflaw.com,"Summer Program, Marshall Wythe School of Law, ...","Trusts and Estates, Real Estate, Trademark Inf...",866-677-2389Phone,"Marquette University, B.A., 1988","Inter-American University of Puerto Rico, J.D....",...,None,None,None,None,None,None,None,None,None,None


In [96]:
### Caso 1: Limpieza practice area
practice_cols = [col for col in df.columns if "practice area" in col.lower() and col != "Areas of Practice"]

# 2. Combinar columnas auxiliares en un string limpio
df['combined_practice'] = df[practice_cols].agg(
lambda row: '; '.join(
        str(val).strip()
        for val in row
        if val and str(val).strip().lower() != 'none'),
    axis=1)

# 3. Usar valor combinado solo si no está vacío
df['Areas of Practice'] = df.apply(
    lambda row: row['combined_practice'] if row['combined_practice'] else row['Areas of Practice'],
    axis=1)

# 4. Limpiar y estandarizar el texto final
df['Areas of Practice'] = (
    df['Areas of Practice']
    .fillna('')                                  # manejar posibles NaN
    .str.replace(',', ';')                       # reemplazar , por ;
    .str.replace(r'\s*;\s*', '; ', regex=True)   # limpiar espacios
    .str.strip('; ')                             # quitar ; sobrantes
    .apply(lambda x: '; ' + x if x and not x.startswith(';') else x))

# 5. Eliminar columnas auxiliares
df.drop(columns=practice_cols + ['combined_practice'], inplace=True)
df.head(5)

,url_martindale,name,firm,url_firm_martindale,website_firm,Biography,Areas of Practice,Contact Information,University Attended,Law School Attended,...,profile_peer_review_star,profile_peer_review_count,Minority Owned,Special Agencies,Military,Hobbies & interests,Transactions,Reported Cases,website,Affiliations
0,https://www.martindale.com/attorney/tessie-lea...,"Tessie Leal-Garabis, Esq.",Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Tessie Leal Garabis is a trial attorney with o...,; Trial Practice; Civil Appeals; Business Law;...,787-650-6994Phone787-767-2645Faxwww.mellado.co...,"Bently University, B.A., 1995","Pontifical Catholic University of Puerto Rico,...",...,None,None,None,None,None,None,None,None,None,None
1,https://www.martindale.com/attorney/ramon-mell...,Ramón Mellado-González,Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Ramón has been an attorney and notary in Puert...,; Complex Business Litigation; Banks and Banki...,787-650-6994Phone787-767-2645Faxwww.mellado.co...,"University of Puerto Rico, B.A., Political Sci...","University of Puerto Rico, LL.B., 1958",...,None,None,None,None,None,None,None,None,None,None
2,https://www.martindale.com/attorney/edgardo-j-...,"Edgardo J. Hernández Ohárriz, Esq.","Hernandez - Oharriz & Santiago Law Firm, P.S.C.",https://www.martindale.com/organization/hernan...,http://www.hoslawfirm.com,Partner/co-founder of Hernandez-Oharriz & Sant...,; Personal Injury; Pedestrian Injuries; Plaint...,855-789-8177Phone787-775-1672Faxwww.hoslawfirm...,"University of Puerto Rico, B.B.A., 2007; Unive...","Inter-American University, J.D., 2010",...,None,None,None,None,None,None,None,None,None,None
3,https://www.martindale.com/attorney/jaime-j-ap...,Jaime J. Aponte-Parsi,"Jaime Aponte-Parsi Law Offices, P.S.C.",https://www.martindale.com/organization/jaime-...,http://www.aponte-parsi.com,Former Partner: Maymi & Aponte-Parsi (2000-200...,; Corporate Law; Taxation; Real Estate; Estate...,877-792-1046Phone787-622-7669Faxwww.aponte-par...,"Boston College, B.A., Economics and Political ...","Boston College, J.D., 1978",...,None,None,None,None,None,None,None,None,None,None
4,https://www.martindale.com/attorney/vicente-a-...,Vicente A. Sequeda-Torres,"Reichard & Calaf, P.S.C.",https://www.martindale.com/organization/reicha...,http://reichardcalaflaw.com,"Summer Program, Marshall Wythe School of Law, ...",; Trusts and Estates; Real Estate; Trademark I...,866-677-2389Phone,"Marquette University, B.A., 1988","Inter-American University of Puerto Rico, J.D....",...,5.0,1,None,None,None,None,None,None,None,None


In [91]:
df.to_excel("Martindale_attorneys_clean.xlsx",index=False)
df.head()

,url_martindale,name,firm,url_firm_martindale,website_firm,Biography,Areas of Practice,Education & Credentials,Contact Information,University Attended,...,profile_peer_review_star,profile_peer_review_count,Minority Owned,Special Agencies,Military,Hobbies & interests,Transactions,Reported Cases,website,Affiliations
0,https://www.martindale.com/attorney/tessie-lea...,"Tessie Leal-Garabis, Esq.",Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Tessie Leal Garabis is a trial attorney with o...,; Trial Practice; Civil Appeals; Business Law;...,,787-650-6994Phone787-767-2645Faxwww.mellado.co...,"Bently University, B.A., 1995",...,None,None,None,None,None,None,None,None,None,None
1,https://www.martindale.com/attorney/ramon-mell...,Ramón Mellado-González,Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Ramón has been an attorney and notary in Puert...,; Complex Business Litigation; Banks and Banki...,,787-650-6994Phone787-767-2645Faxwww.mellado.co...,"University of Puerto Rico, B.A., Political Sci...",...,None,None,None,None,None,None,None,None,None,None
2,https://www.martindale.com/attorney/edgardo-j-...,"Edgardo J. Hernández Ohárriz, Esq.","Hernandez - Oharriz & Santiago Law Firm, P.S.C.",https://www.martindale.com/organization/hernan...,http://www.hoslawfirm.com,Partner/co-founder of Hernandez-Oharriz & Sant...,; Personal Injury; Pedestrian Injuries; Plaint...,,855-789-8177Phone787-775-1672Faxwww.hoslawfirm...,"University of Puerto Rico, B.B.A., 2007; Unive...",...,None,None,None,None,None,None,None,None,None,None
3,https://www.martindale.com/attorney/jaime-j-ap...,Jaime J. Aponte-Parsi,"Jaime Aponte-Parsi Law Offices, P.S.C.",https://www.martindale.com/organization/jaime-...,http://www.aponte-parsi.com,Former Partner: Maymi & Aponte-Parsi (2000-200...,; Corporate Law; Taxation; Real Estate; Estate...,,877-792-1046Phone787-622-7669Faxwww.aponte-par...,"Boston College, B.A., Economics and Political ...",...,None,None,None,None,None,None,None,None,None,None
4,https://www.martindale.com/attorney/vicente-a-...,Vicente A. Sequeda-Torres,"Reichard & Calaf, P.S.C.",https://www.martindale.com/organization/reicha...,http://reichardcalaflaw.com,"Summer Program, Marshall Wythe School of Law, ...",; Trusts and Estates; Real Estate; Trademark I...,,866-677-2389Phone,"Marquette University, B.A., 1988",...,5.0,1,None,None,None,None,None,None,None,None


In [7]:
links_revisados = data_for_PA
for entry in data_for_PA:
    link = entry.get("url_martindale", "")
print(link)

https://www.martindale.com/attorney/samuel-garcia-angeli-1571737/
